In [1]:
import sys; sys.path.append('..')
from adjective_reading.parsing import *
import networkx as nx


OK


In [107]:
df = pd.read_csv('../data/keyword_sent_corpus_parsed.csv.gz').fillna('').drop_duplicates(subset=['url','context1'])
df

,url,page_num,sent_num,sent,context0,context1,context2,token_num,token0,token1,...,id,text,deprel,relation,head_idx,head_text,head_pos,head_lemma,prep,cc
0,http://www.jstor.org/stable/461288,4,4,We are all our lifetime __reading__ the copiou...,194 Pascal et le d6s6quilibre ment cette circu...,We are all our lifetime reading the copious se...,"One moral we have already deduced, in consider...",5,lifetime,reading,...,6,reading,acl,acl,5,lifetime,NOUN,lifetime,,
2,http://www.jstor.org/stable/459148,1,6,It is this conflict that is responsible for th...,"""4 Had Shelley composed the poem in tranquilit...",It is this conflict that is responsible for th...,"The real difficulty in reading ""Mont Blanc"" is...",22,cult,reading,...,25,reading,amod,amod,22,such,ADJ,such,,
7,http://www.jstor.org/stable/459148,2,2,The poet's ambiguous attitude towards his subj...,I. J. Kapstein last three lines; here with a s...,The poet's ambiguous attitude towards his subj...,,13,in,reading,...,15,reading,mark,mark,14,in,SCONJ,in,,
11,http://www.jstor.org/stable/459148,5,10,"By this __reading__, therefore, Shelley is say...","But if ""passively"" is taken as qualifying ""now...","By this reading, therefore, Shelley is saying ...",The idea of the passivity of mind in knowledge...,2,this,reading,...,3,reading,det,det,2,this,DET,this,,
14,http://www.jstor.org/stable/459148,13,6,"Whatever the __reading__, one sees, neverthele...",For the anti-climax of the poem contradicts th...,"Whatever the reading, one sees, nevertheless, ...","The essay ""On Life"" reveals with what relief S...",2,the,reading,...,3,reading,det,det,2,the,DET,the,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30824,http://www.jstor.org/stable/3716792,1,24,This book thus provides convincing evidence in...,The first section of the book is devoted to a ...,This book thus provides convincing evidence in...,Word-lists are appended to each passage: thoug...,26,prose,reading,...,27,reading,amod,amod,25,continuous,ADJ,continuous,,
30827,http://www.jstor.org/stable/2872184,3,9,Nonetheless he is different from the narrator ...,The only difference is that this character's w...,Nonetheless he is different from the narrator ...,For all the changes in tone of Wichita Vortex ...,11,in,reading,...,13,reading,compound,compound,14,lyric,NOUN,lyric,,
30828,http://www.jstor.org/stable/462592,2,6,In probing the gender implications of various ...,"In contrast to theorists of patriar- chy, whos...",In probing the gender implications of various ...,This deuniversalization and denaturalization o...,22,of,reading,...,27,reading,mark,mark,26,of,SCONJ,of,,
30832,http://www.jstor.org/stable/3727574,1,20,The book seems to me to play down rather too m...,To demonstrate it one would have to give rathe...,The book seems to me to play down rather too m...,Mann saw his task as 'tief erkennen und schin ...,17,of,reading,...,20,reading,mark,mark,19,of,SCONJ,of,,


In [ ]:
G = nx.Graph()

def add_node(G, node, **data):
    if node not in G:
        G.add_node(node, **data)
    else:
        G.nodes[node].update(data)

def add_edge(G, node1, node2, **data):
    if node1 not in G:
        add_node(G, node1)
    if node2 not in G:
        add_node(G, node2)
    if G.has_edge(node1, node2):
        G[node1][node2]['weight'] += 1
        # G[node1][node2]['data'].append(data)
    else:
        G.add_edge(node1, node2, weight=1, **data)

for i,row in df.iterrows():
    node1 = row['lemma']
    node1_d = {'text': row['lemma'], 'pos': row['pos']}
    node2 = row['head_lemma']
    node2_d = {'text': row['head_lemma'], 'pos': row['head_pos']}
    add_node(G, node1, **node1_d)
    add_node(G, node2, **node2_d)


    add_edge(G, node1, node2, **row.to_dict())


# remove edges with weight < 2
G.remove_edges_from([e for e in G.edges() if G[e[0]][e[1]]['weight'] < 2])
# remove isolated nodes
G.remove_nodes_from([n for n in G.nodes() if G.degree(n) == 0])

In [121]:
# list(G.edges(data=True))

In [122]:
G.order(), G.size()

(402, 453)

In [123]:
'surface' in list(G.nodes())

True

In [125]:
nx.write_graphml(G, '../data/keyword_sent_corpus_parsed.graphml')